# Context-Aware Step Counting
This project implements a step-counting algorithm that dynamically adjusts based on activity contexts (e.g., walking, jogging, stairs). The project also evaluates accelerometer and gyroscope data individually and combined, using key metrics and visualizations to highlight findings.

## 1. Problem Definition
Traditional step-counting algorithms often fail in diverse activity contexts. This project aims to:
- Classify activities using sensor data (accelerometer, gyroscope).
- Adjust step-counting parameters dynamically based on detected activity.

### Approach
- Use accelerometer and gyroscope data for classification.
- Dynamically adapt step-counting thresholds for each activity.
- Validate improvements over baseline step counting.

### How It Extends Course Assignments
- Incorporates multi-sensor fusion (accelerometer + gyroscope).
- Introduces dynamic parameterization for activity-aware step counting.

## 2. Data Collection & Processing

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, find_peaks
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

# Visualization Settings
plt.rcParams['figure.figsize'] = [9, 6]
plt.rcParams['axes.grid'] = True

# Helper: Timestamp Transformation
def transform_time_to_datetime(root):
    """Convert timestamps in collected data to datetime."""
    for activity in os.listdir(root):
        activity_path = os.path.join(root, activity)
        for file in os.listdir(activity_path):
            if file.endswith('.csv'):
                file_path = os.path.join(activity_path, file)
                df = pd.read_csv(file_path)
                df['time'] = pd.to_datetime(df['time'], unit='ns', errors='coerce')
                df.to_csv(file_path, index=False)

# Transform timestamps for MyData directory
transform_time_to_datetime('./data/MyData')

### Preprocessing and Feature Engineering

In [2]:
# Helper: Signal Preprocessing
def calc_magnitude(df, x_col, y_col, z_col):
    """Calculate the magnitude of a 3-axis signal."""
    df['magnitude'] = np.sqrt(df[x_col]**2 + df[y_col]**2 + df[z_col]**2)
    df['magnitude'] -= df['magnitude'].mean()
    return df

def remove_noise(df, column='magnitude', cutoff=5, sample_rate=50):
    """Apply a low-pass Butterworth filter to reduce noise."""
    nyquist = 0.5 * sample_rate
    normalized_cutoff = cutoff / nyquist
    b, a = butter(2, normalized_cutoff, btype='low', analog=False)
    df[f'filtered_{column}'] = filtfilt(b, a, df[column])
    return df

# Example: Load walking data
data = pd.read_csv('./data/MyData/walking.csv')
data = calc_magnitude(data, 'x', 'y', 'z')
data = remove_noise(data)

# Plot filtered signal
plt.plot(data['filtered_magnitude'][:1000], label='Filtered Magnitude')
plt.title('Filtered Accelerometer Signal (Walking)')
plt.legend()
plt.show()

## 3. Feature Extraction & Sensor Comparison

In [3]:
# Helper: Extract Features
def extract_features(window):
    """Extract statistical features from a signal window."""
    features = {
        'mean': window.mean(),
        'std': window.std(),
        'max': window.max(),
        'min': window.min(),
        'q25': window.quantile(0.25),
        'q75': window.quantile(0.75)
    }
    return pd.DataFrame([features])

# Helper: Sliding Window
def sliding_window_features(data, column, window_size=50):
    """Apply sliding window to extract features from data."""
    features_list = []
    for start in range(0, len(data) - window_size + 1, window_size):
        window = data[column][start:start + window_size]
        features = extract_features(window)
        features_list.append(features)
    return pd.concat(features_list, ignore_index=True)

# Example: Compare Accelerometer vs. Gyroscope
accel_features = sliding_window_features(data, 'filtered_magnitude')
gyro_data = pd.read_csv('./data/MyData/walking_gyro.csv')
gyro_data = calc_magnitude(gyro_data, 'gyro_x', 'gyro_y', 'gyro_z')
gyro_data = remove_noise(gyro_data)
gyro_features = sliding_window_features(gyro_data, 'filtered_magnitude')

# Visualization
plt.plot(data['filtered_magnitude'][:500], label='Accelerometer')
plt.plot(gyro_data['filtered_magnitude'][:500], label='Gyroscope')
plt.legend()
plt.title('Comparison: Accelerometer vs Gyroscope Signal')
plt.show()

## 4. Classification and Step Counting

In [4]:
# Combine Features
combined_features = pd.concat([accel_features, gyro_features], axis=1)

# Add Labels
combined_features['label'] = 'walking'

# Encode Labels
le = LabelEncoder()
combined_features['label_encoded'] = le.fit_transform(combined_features['label'])

# Train-Test Split
X = combined_features.drop(columns=['label', 'label_encoded'])
y = combined_features['label_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

## 5. Visualize Results
- Compare accelerometer-only, gyroscope-only, and combined data.

In [5]:
# Confusion Matrix for Combined Sensors
cm = confusion_matrix(y_test, y_pred)
plt.imshow(cm, cmap='Blues', interpolation='nearest')
plt.colorbar()
plt.title('Confusion Matrix: Combined Sensors')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()